## Classificação de números através de áudios
O algorimo tem como objetivo desenvolver um modelo que consiga classificar números através de entrada de áudio, para isso foi usado o dataset [__MNIST__](https://www.kaggle.com/datasets/sripaadsrinivasan/audio-mnist). Basicamente o dataset contém __30000 áudios__ com o som dos números de 0 até 9 falados por 60 diferentes pessoas. 

Para a resolução do problema foi escolhido o uso de __Redes Neurais Artificiais__, ao final do processo será possível entrar com um áudio e o modelo classificará e informará qual o número dito.

### Estrutura do projeto

<ul>
  <li>📂 <strong>dataset</strong>
    <ul>
      <li>📂 <strong>audios</strong>: Pasta com os áudios do dataset, extraia os áudios para essa pasta.</li>
      <li>📂 <strong>audios_extras</strong>: Pasta com os áudios extras, gravados manualmente por mim para testar o modelo.</li>
      <li>📂 <strong>audios_processed</strong>: Pasta com os áudios processados em csv caso você não queira baixar o dataset, também já está separado em dados de treino e teste.</li>
    </ul>
  </li>
  <li>.gitignore</li>
  <li>audio_processing.ipynb</li>
</ul>

__OBS: O dataset não está incluso no repositório devido a limites do github, faça o download do dataset [__aqui__](https://www.kaggle.com/datasets/sripaadsrinivasan/audio-mnist). Em seguida extraia os áudios para a pasta "/dataset/audios/..." ficando como mostrado abaixo:__

<ul>
  <li>📂 <strong>dataset</strong>
    <ul>
      <li>📂 <strong>audios</strong>
      <ul>
        <li>📂 <strong>01</strong></li>
        <li>📂 <strong>02</strong></li>
        <li>📂 <strong>03</strong></li>
        <li>📂 <strong>...</strong></li>
      </ul>
    </li>
    </ul>
  </li>
</ul>

##### 1. Bibliotecas usadas

Foram usadas bibliotecas padrões para manipulação de dados como __pandas e numpy__, para a manipulação de entradas de áudio foi usado as libs __glob e librosa__, por fim, usou-se bibliotecas para desenvolvimento do modelo como __sklearn e keras__.

In [249]:
import pandas as pd
import numpy as np

from glob import glob

import librosa
import librosa.display
import IPython.display as ipd

import os

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import ModelCheckpoint

##### 2. Funções utilitárias

Foram definidas algumas funções básicas para ajudar no desenvolvimento.

__describe_audio__: Fornece um resumo do áudio, exibindo um reprodutor e o gráfico do mesmo.<br>
__get_audio_mfcc__: Retorna o Mel-frequency cepstral coeficiente do áudio.<br>
__get_audio_class__: Retorna qual a classe do áudio.<br>
__extract_audio_data__: Extrai de um array de áudios os dados do áudio, retornando o data cru e o data pronto para o modelo.
__extract_audio_target__: Extrai de um array de áudios a classe dos mesmos, retornando as classes cruas e as classes em dummy encoder.

In [250]:
def describe_audio(audio):
    y, sr = librosa.load(audio)
    pd.Series(y).plot()
    return ipd.Audio(audio)

def get_audio_mfcc(audio, n_mfcc=50):
    y, sr = librosa.load(audio)
    mfcc = librosa.feature.mfcc(y, sr, n_mfcc=n_mfcc)
    # mfcc_scaled = np.mean(mfcc.T, axis=0)
    return mfcc

def get_audio_class(audio):
    filename = os.path.basename(audio.title())
    return int(filename.split('_')[0])

def extract_audio_data(audios):
    n_mfcc = 50
    raw_data = []
    transformed_data = np.empty((0, 50))

    for audio_index in range(0, len(audios)):
        print('Extracting data from audio... ' + str(audio_index+1) + '/' + str(len((audios)))) 
        raw_data.append(get_audio_mfcc(audios[audio_index], n_mfcc))

    for mfcc in raw_data:
        mfcc_transposed = np.mean(mfcc.T, axis=0)
        transformed_data = np.vstack((transformed_data, mfcc_transposed.tolist()))

    return raw_data, transformed_data

def extract_audio_target(audios):
    raw_target = []

    for audio_index in range(0, len(audios)):
        print('Extracting target from audio... ' + str(audio_index+1) + '/' + str(len((audios)))) 
        raw_target.append(get_audio_class(audios[audio_index]))

    transformed_target = pd.get_dummies(raw_target).values

    return raw_target, transformed_target

##### 3. Carregamento dos áudios que serão usados para treino e teste.

In [251]:
audio_files = glob('dataset/audios/*/*.wav')
len(audio_files)

30000

##### 4. Transformação dos áudios

Será gerado nesse passo os previsores e as classes que serão usados para o treino e teste do modelo. Os previsores (x), são extraídos dos áudios usando o __Mel-frequency cepstral__ coeficiente, basicamente esse coeficiente consegue resumir o dado em suas características mais importantes do ponto de vista do modelo. Já as classes são extraídas dos nomes dos arquivos, os primeiros caracteres antes do _ (underline) significa qual a classe do áudio, por exemplo o áudio __"05_01_02.wav"__, o __05__ inicial siginifica que esse áudio é o som do número cinco.

Para treinar o modelo, usaremos as variáveis com o prefixo __"transformed..."__, pois elas já estão no formato que o modelo irá receber.

__OBS: Esse processo pode levar alguns minutos...__

In [252]:
raw_data, transformed_data = extract_audio_data(audio_files)
raw_target, transformed_target = extract_audio_target(audio_files)

Extracting data from audio... 1/30000
Extracting data from audio... 2/30000
Extracting data from audio... 3/30000
Extracting data from audio... 4/30000
Extracting data from audio... 5/30000
Extracting data from audio... 6/30000
Extracting data from audio... 7/30000
Extracting data from audio... 8/30000
Extracting data from audio... 9/30000
Extracting data from audio... 10/30000
Extracting data from audio... 11/30000
Extracting data from audio... 12/30000
Extracting data from audio... 13/30000
Extracting data from audio... 14/30000
Extracting data from audio... 15/30000
Extracting data from audio... 16/30000
Extracting data from audio... 17/30000
Extracting data from audio... 18/30000
Extracting data from audio... 19/30000
Extracting data from audio... 20/30000
Extracting data from audio... 21/30000
Extracting data from audio... 22/30000
Extracting data from audio... 23/30000
Extracting data from audio... 24/30000
Extracting data from audio... 25/30000
Extracting data from audio... 26/3

##### 5. Divisão dos dados

Nesse passo usaremos o __train_test_split__ para realizar a divisão randômica entre a base de dados, separando os dados uma parte para treino, outra parte para teste... por padrão foi definido a divisão __70/30__, onde __70%__ dos dados serão usados para treino e __30%__ usados para teste.

In [255]:
training_data, test_data, training_target, test_target = train_test_split(transformed_data, transformed_target, test_size=0.3, random_state=0)

print(training_data.shape, test_data.shape, training_target.shape, test_target.shape)

(21000, 50) (9000, 50) (21000, 10) (9000, 10)


##### 6. Criação do modelo

Criou-se o modelo, definido em 4 camadas incluindo a camada de saída, a camada de entrada contém 50 perceptrons, pois cada áudio foi mapeado em 50 colunas. Adicionamos dropout para evitar o overfitting, e usou-se a função de loss do tipo __categorical_crossentropy__.

A camada de saída apresenta 10 perceptrons devido a saída ser 10 classes, indo de 0 até 9, e as classes foram mapeadas em one hot encoder, gerando assim 10 colunas de saída.

In [256]:
model = Sequential()

model.add(Dense(units=100, activation='relu', input_dim=50))
model.add(Dropout(0.2))

model.add(Dense(units=100, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(units=100, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(units=10, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 100)               5100      
                                                                 
 dropout_6 (Dropout)         (None, 100)               0         
                                                                 
 dense_9 (Dense)             (None, 100)               10100     
                                                                 
 dropout_7 (Dropout)         (None, 100)               0         
                                                                 
 dense_10 (Dense)            (None, 100)               10100     
                                                                 
 dropout_8 (Dropout)         (None, 100)               0         
                                                                 
 dense_11 (Dense)            (None, 10)               

##### 6. Variáveis e treinamento

Definimos que o modelo iria realizar __100 epochs__ e efetuar o balanceamento dos pesos a cada __32 items__... adicionamos também um _check pointer_ para salvar o melhor modelo entre os 100 ciclos. Em seguida, o modelo será treinado.

In [257]:
num_epochs = 100
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='saved_models/audio_classification.hdf5', 
                               save_best_only=True)

model.fit(training_data, training_target, 
          epochs=num_epochs,
          validation_data=(test_data, test_target), 
          callbacks=[checkpointer])

Epoch 1/100
657/657 [==============================] - 3s 3ms/step - loss: 4.1002 - accuracy: 0.1202 - val_loss: 2.1866 - val_accuracy: 0.1820
Epoch 2/100
657/657 [==============================] - 2s 3ms/step - loss: 2.1330 - accuracy: 0.1859 - val_loss: 2.0414 - val_accuracy: 0.2040
Epoch 3/100
657/657 [==============================] - 2s 3ms/step - loss: 1.9891 - accuracy: 0.2377 - val_loss: 1.7647 - val_accuracy: 0.2903
Epoch 4/100
657/657 [==============================] - 2s 3ms/step - loss: 1.7608 - accuracy: 0.2979 - val_loss: 1.5677 - val_accuracy: 0.3809
Epoch 5/100
657/657 [==============================] - 2s 3ms/step - loss: 1.4786 - accuracy: 0.4020 - val_loss: 1.1010 - val_accuracy: 0.5789
Epoch 6/100
657/657 [==============================] - 2s 3ms/step - loss: 0.9061 - accuracy: 0.6524 - val_loss: 0.5359 - val_accuracy: 0.8126
Epoch 7/100
657/657 [==============================] - 2s 3ms/step - loss: 0.6000 - accuracy: 0.7826 - val_loss: 0.3847 - val_accuracy: 0.8583

##### 6. Teste de precisão

Ao fim do treinamento, rodamos o modelo com os dados separados de teste para fazer uma avaliação da sua performace.

In [268]:
accuracy = model.evaluate(test_data, test_target, verbose = 0)
print('Precisão do modelo é de ' + str(accuracy[1]*100) + '%')

Precisão do modelo é de 96.4555561542511%


##### 7. Teste manual

Na pasta _dataset/audios_extras_ foi adicionado alguns exemplos de áudios gravados por mim, para testar a eficiência do modelo. Você também pode tentar, basta o áudio ser um arquivo __.wav__ e ter no máximo __1 segundo__, lembrando que o modelo foi treinado com dados limpos e padronizados do dataset usado ([__MNIST__](https://www.kaggle.com/datasets/sripaadsrinivasan/audio-mnist)), então caso você coloque um áudio muito grande, muito baixo, ou muito alto, provavelmente o modelo não irá reconhecer. O objetivo desse algoritmo não é ser o melhor, mas sim didático.

In [277]:
test_audio_files = glob('dataset/audios_extras/*.wav')
len(test_audio_files)

_, test_transformed_data = extract_audio_data(test_audio_files)

manual_test_result = model.predict(test_transformed_data)
manual_test_result = np.where(manual_test_result > 0.5, 1, 0)

for audio_index in range(0, len(test_audio_files)):
    print('O áudio "' + str(test_audio_files[audio_index].title()) + '" foi classificado como ' + str(np.argmax(manual_test_result[audio_index])))

Extracting data from audio... 1/9
Extracting data from audio... 2/9
Extracting data from audio... 3/9
Extracting data from audio... 4/9
Extracting data from audio... 5/9
Extracting data from audio... 6/9
Extracting data from audio... 7/9
Extracting data from audio... 8/9
Extracting data from audio... 9/9
1/1 [==============================] - 0s 22ms/step
O áudio "Dataset/Audios_Extras\01.Wav" foi classificado como 1
O áudio "Dataset/Audios_Extras\02.Wav" foi classificado como 2
O áudio "Dataset/Audios_Extras\03.Wav" foi classificado como 3
O áudio "Dataset/Audios_Extras\04.Wav" foi classificado como 4
O áudio "Dataset/Audios_Extras\05.Wav" foi classificado como 5
O áudio "Dataset/Audios_Extras\06.Wav" foi classificado como 6
O áudio "Dataset/Audios_Extras\07.Wav" foi classificado como 7
O áudio "Dataset/Audios_Extras\08.Wav" foi classificado como 8
O áudio "Dataset/Audios_Extras\09.Wav" foi classificado como 9
